In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
from dotenv import load_dotenv
load_dotenv()

import os
bearer_token = os.environ.get("TOKEN")

In [3]:
os.environ['TOKEN'] = 'ENTER BEARER TOKEN HERE'

In [4]:
def auth():
    return os.environ.get("TOKEN")

In [5]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

In [6]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [7]:
def create_url(keyword, start_date, end_date, next_token):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword, 
                    'start_time' : start_list[i], 
                    'end_time' : end_list[i],
                    'max_results' : max_results,
                    'tweet.fields': 'created_at',
                    'next_token': {}
                     }
    
    return (search_url, query_params)

In [8]:
def connect_to_endpoint(url, headers, params, next_token=None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.get( url, auth=bearer_oauth, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [9]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that
        #Time created
        created_at = dateutil.parser.parse(tweet['created_at'])
        #Tweet ID
        tweet_id = tweet['id']
        #Tweet text
        text = tweet['text']
        # Assemble all data in a list
        res = [created_at, tweet_id, text]
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter)

In [10]:
# Load start and end times
time_range = pd.read_csv("TwitterAPI_start_end_times.csv")
start_list = time_range['start_time'].tolist()
end_list = time_range['end_time'].tolist()

In [11]:
#Inputs for tweets
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "tesla lang:en -from:tesla -is:retweet"
start_list = start_list
end_list = end_list

max_results = 100

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("API_data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['created_at','id','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 200 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i], end_list[i], next_token)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "API_data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "API_data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8qu9uxdqqcwqqd6nchxvrbqfel
Start Date:  2021-11-09T00:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  100
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv8qu9uxdqqcwqqd6nchxvrbqfel
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8qu9uvv464v9ztefuzutf99ail
Start Date:  2021-11-09T00:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  200
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8qua5vxix070i7ea4smtybecfx
Start Date:  2021-11-09T01:00:00.000Z
# of Tweets added from this response:  99
Total # of Tweets added:  299
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv8qua5vxix070i7ea4smtybecfx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv8qua5ud757v0ivr35s4drtjfy5
Start Date:  2021-11-09T01:00:00.000Z
# of Twe

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95ite2z09qretjdujqnp01sgal
Start Date:  2021-11-09T12:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  3089
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv95ite2z09qretjdujqnp01sgal
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95itdzx4goqhykar6gz3nfz98d
Start Date:  2021-11-09T12:00:00.000Z
# of Tweets added from this response:  98
Total # of Tweets added:  3187
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv95itdzx4goqhykar6gz3nfz98d
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95itdwwisx16mb7cksaazox4e5
Start Date:  2021-11-09T12:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  3287
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95itzhqmnnvryv7qyflv80kx31
Start Date:  2021-11-09T13:00:00.000Z
# of 

-------------------
Token:  b26v89c19zqg8o3fpdv95n28vrxo9wi9c3pkvqr5enwxp
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95n28sqwi1e63sfdsmyyqz8br1
Start Date:  2021-11-10T00:00:00.000Z
# of Tweets added from this response:  99
Total # of Tweets added:  6080
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv95n28sqwi1e63sfdsmyyqz8br1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95n28pqxpebhhujo54jzdrs22l
Start Date:  2021-11-10T00:00:00.000Z
# of Tweets added from this response:  99
Total # of Tweets added:  6179
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95n2u7knjtzuswbq4jevh8w399
Start Date:  2021-11-10T01:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  6279
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv95n2u7knjtzuswbq4jevh8w399
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95n2jsrdyt0421ja1q6qmbz88t
Sta

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95rc00fk6sezicul6t870a7o8t
Start Date:  2021-11-10T14:00:00.000Z
# of Tweets added from this response:  99
Total # of Tweets added:  9076
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv95rc00fk6sezicul6t870a7o8t
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95rbzywq7qxm0h6c9ix09u8wot
Start Date:  2021-11-10T14:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  9176
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv95rbzywq7qxm0h6c9ix09u8wot
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95rbzxdoagd0uw60xt7okg743h
Start Date:  2021-11-10T14:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  9276
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95rcldp722ty90t7y2vffarjp9
Start Date:  2021-11-10T15:00:00.000Z
# of 

-------------------
Token:  b26v89c19zqg8o3fpdv95vkurqxgrzqjmoxcyrohm5xfh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95vkuq7o1haphkjmq60w1y09kt
Start Date:  2021-11-11T02:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  12069
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95vlg6i4b6bxi41r1gfgkk73wd
Start Date:  2021-11-11T03:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  12169
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv95vlg6i4b6bxi41r1gfgkk73wd
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95vlg3i5ozysbim11eba0tevzx
Start Date:  2021-11-11T03:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  12269
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv95vlr6ilxc2k91rlx0ltebin3x
Start Date:  2021-11-11T04:00:00.000Z
#

...

-------------------
Token:  b26v89c19zqg8o3fpdv9koeqc4ss6t2r4y3pfpirhdu9p
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9koeq92a1trv9cds8gs67a2v7h
Start Date:  2021-11-12T05:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  18060
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9kof1f63rovzl1a853t9odgnel
Start Date:  2021-11-12T06:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  18160
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv9kof1f63rovzl1a853t9odgnel
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9kof1c3sjek5440zmf2got769p
Start Date:  2021-11-12T06:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  18260
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9kofmvfx7erb4hksawrcfetxml
Start Date:  2021-11-12T07:00:00.000Z
#

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9ksp31kttljrdfwgbaw1apnvr1
Start Date:  2021-11-12T20:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  21057
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv9ksp31kttljrdfwgbaw1apnvr1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9ksp301zrb3s7xcgp5y04ey64d
Start Date:  2021-11-12T20:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  21157
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9kspe1l3r8c039omlshgqdif7h
Start Date:  2021-11-12T21:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  21257
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv9kspe1l3r8c039omlshgqdif7h
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9kspe01ue1a20c0lpe9xsokvwd
Start Date:  2021-11-12T21:00:00.000Z
#

-------------------
Token:  b26v89c19zqg8o3fpdv9kwyu35xfax5g90efe7fejc2nx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9kwyjld4rlfwin7ys9ph5qvddp
Start Date:  2021-11-13T10:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  24055
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9kwz566lkreuw0dk550rlszrst
Start Date:  2021-11-13T11:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  24155
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv9kwz566lkreuw0dk550rlszrst
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9kwz51o850z6xx4h83v0hkqohp
Start Date:  2021-11-13T11:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  24255
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9kz1gsi8fybydzq0u9e2zo5oql
Start Date:  2021-11-13T12:00:00.000Z
#

-------------------
Token:  b26v89c19zqg8o3fpdv9l18aaryzeuqvpwsa6h8ijb7ul
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9l17zuhsdl9a3rqtfwpfqknkot
Start Date:  2021-11-13T23:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  27050
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9l18lasgburljrnqfkgdzj2uf1
Start Date:  2021-11-14T00:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  27150
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv9l18lasgburljrnqfkgdzj2uf1
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9l18l68srv2m0aksccwf25dsvx
Start Date:  2021-11-14T00:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  27250
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9l3awx43ceapyp8ic3skseplz1
Start Date:  2021-11-14T01:00:00.000Z
#

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9l5i1fdvwbp0dqoasc2t9rl9bx
Start Date:  2021-11-14T13:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  30046
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv9l5i1fdvwbp0dqoasc2t9rl9bx
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9l5hqz2upsoqig2az7omnwe16l
Start Date:  2021-11-14T13:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  30146
-------------------
-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9l5icffnnsiu5owydigezc1xfh
Start Date:  2021-11-14T14:00:00.000Z
# of Tweets added from this response:  100
Total # of Tweets added:  30246
-------------------
-------------------
Token:  b26v89c19zqg8o3fpdv9l5icffnnsiu5owydigezc1xfh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpdv9l5icdvr1gje3uscwu7sbh8dbx
Start Date:  2021-11-14T14:00:00.000Z
#

In [12]:
# API for tweet counts
search_url = "https://api.twitter.com/2/tweets/counts/recent"

# Optional params: start_time,end_time,since_id,until_id,next_token,granularity
query_params = {'query': 'tesla lang:en', 'start_time': "2021-11-09T00:00:00.000Z", "end_time": "2021-11-15T00:00:00.000Z",'granularity': 'day'}

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentTweetCountsPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.request("GET", search_url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def main():
    json_response = connect_to_endpoint(search_url, query_params)
    return json_response
    #print(json.dumps(json_response, indent=4, sort_keys=True))

if __name__ == "__main__":
    main()

200


In [13]:
response = main()
response = json.dumps(response, indent=4, sort_keys=True)
print(response)

200
{
    "data": [
        {
            "end": "2021-11-10T00:00:00.000Z",
            "start": "2021-11-09T00:00:00.000Z",
            "tweet_count": 51366
        },
        {
            "end": "2021-11-11T00:00:00.000Z",
            "start": "2021-11-10T00:00:00.000Z",
            "tweet_count": 42847
        },
        {
            "end": "2021-11-12T00:00:00.000Z",
            "start": "2021-11-11T00:00:00.000Z",
            "tweet_count": 57487
        },
        {
            "end": "2021-11-13T00:00:00.000Z",
            "start": "2021-11-12T00:00:00.000Z",
            "tweet_count": 44835
        },
        {
            "end": "2021-11-14T00:00:00.000Z",
            "start": "2021-11-13T00:00:00.000Z",
            "tweet_count": 39768
        },
        {
            "end": "2021-11-15T00:00:00.000Z",
            "start": "2021-11-14T00:00:00.000Z",
            "tweet_count": 61288
        }
    ],
    "meta": {
        "total_tweet_count": 297591
    }
}


In [14]:
x = json.loads(response)['data']
x

[{'end': '2021-11-10T00:00:00.000Z',
  'start': '2021-11-09T00:00:00.000Z',
  'tweet_count': 51366},
 {'end': '2021-11-11T00:00:00.000Z',
  'start': '2021-11-10T00:00:00.000Z',
  'tweet_count': 42847},
 {'end': '2021-11-12T00:00:00.000Z',
  'start': '2021-11-11T00:00:00.000Z',
  'tweet_count': 57487},
 {'end': '2021-11-13T00:00:00.000Z',
  'start': '2021-11-12T00:00:00.000Z',
  'tweet_count': 44835},
 {'end': '2021-11-14T00:00:00.000Z',
  'start': '2021-11-13T00:00:00.000Z',
  'tweet_count': 39768},
 {'end': '2021-11-15T00:00:00.000Z',
  'start': '2021-11-14T00:00:00.000Z',
  'tweet_count': 61288}]

In [15]:
# export count data to csv
import csv 
cols = ["end", "start", "tweet_count"]
path = "API_count_data.csv"
with open(path, 'w') as f: 
    wr = csv.DictWriter(f, fieldnames = cols) 
    wr.writeheader() 
    wr.writerows(x) 